In [2]:
import pyroSAR
from pyroSAR.snap.auxil import parse_recipe, parse_node
from pyroSAR.snap.auxil import gpt
from pyroSAR import identify

In [3]:
#import logging
#logging.basicConfig(level=logging.INFO)

#import os
#import pickle

#os.environ['PATH'] = "C:/Program Files/snap/bin"
#os.environ['PATH2'] = "K:/Rough/output"

#os.environ['PROJ_LIB'] = '/usr/share/proj'

# read1 = os.path.join('K:/Rough/sar data/coherence/S1B_IW_SLC__1SDV_20210814T165951_20210814T170018_028245_035EA1_57A5.zip')
# read2 = os.path.join('K:/Rough/sar data/coherence/S1B_IW_SLC__1SDV_20210826T165952_20210826T170019_028420_036413_214F.zip')
# id_1 = identify(read1).outname_base()
# id_2 = identify(read2).outname_base()
# print(id_1)
# print(id_2)

read1 = identify('K:/Rough/sar data/coherence/S1B_IW_SLC__1SDV_20210814T165951_20210814T170018_028245_035EA1_57A5.zip')
read2 = identify('K:/Rough/sar data/coherence/S1B_IW_SLC__1SDV_20210826T165952_20210826T170019_028420_036413_214F.zip')
print(read1.outname_base())

S1B__IW___A_20210814T165951


In [4]:
%%time

workflow = parse_recipe('blank')

read1 = parse_node('Read')
read1.parameters['file'] = read1
read1.parameters['formatName'] = 'SENTINEL-1'
workflow.insert_node(read1)

read2 = parse_node('Read')
read2.parameters['file'] = read2
read2.parameters['formatName'] = 'SENTINEL-1'
workflow.insert_node(read2)

split1 = parse_node('TOPSAR-split')
split1.parameters['subswath'] = 'IW1'
split1.parameters['selectedPolarisations'] = 'VV'
split1.parameters['firstBurstIndex'] = 6
split1.parameters['lastBurstIndex'] = 7
workflow.insert_node(split1, before=read1.id)

split2 = parse_node('TOPSAR-split')
split2.parameters['subswath'] = 'IW1'
split2.parameters['selectedPolarisations'] = 'VV'
split2.parameters['firstBurstIndex'] = 6
split2.parameters['lastBurstIndex'] = 7
workflow.insert_node(split2, before=read2.id)

orb1 = parse_node('Apply-Orbit-File')
orb1.parameters['orbitType'] = 'Sentinel Precise (Auto Download)'
orb1.parameters['continueOnFail'] = True
workflow.insert_node(orb1, before=split1.id)

orb2 = parse_node('Apply-Orbit-File')
orb2.parameters['orbitType'] = 'Sentinel Precise (Auto Download)'
orb2.parameters['continueOnFail'] = True
workflow.insert_node(orb2, before=split2.id)

stack = parse_node('Back-Geocoding')
stack.parameters['demName'] = 'SRTM 1Sec Grid'
stack.parameters['demResamplingMethod'] = 'BILINEAR_INTERPOLATION'
stack.parameters['resamplingType'] = 'BILINEAR_INTERPOLATION'
stack.parameters['maskOutAreaWithoutElevation'] = True
stack.parameters['outputDerampDemodPhase'] = True
workflow.insert_node(stack, before= [orb2.id, orb1.id])

esd = parse_node('Enhanced-Spectral-Diversity')
workflow.insert_node(esd, before=stack.id)

# ifg = parse_node('Interferogram')
# ifg.parameters['subtractFlatEarthPhase'] = True
# ifg.parameters['srpPolynomialDegree'] = 5
# ifg.parameters[''] =
# ifg.parameters[''] =
# ifg.parameters[''] =
# ifg.parameters[''] =
# ifg.parameters[''] =
# ifg.parameters[''] =
# ifg.parameters[''] =
# ifg.parameters[''] =
# ifg.parameters[''] =

# deburst
deb = parse_node("TOPSAR-Deburst")
workflow.insert_node(deb, before = esd.id)

#merge_list.append(deb.id)


Wall time: 68 ms


In [5]:
write = parse_node("Write")
write.parameters["file"] = "K:/Rough/output/pysar22"
write.parameters["formatName"] = "BEAM-DIMAP"
workflow.insert_node(write, before = deb.id)

workflow.write("K:/Rough/output/pyro11_workflow1")

#xmlname = 'pyro11_workflow1'
#workflow.write('K:/Rough/output/'+xmlname)


#gpt(xmlfile='K:/Rough/output/'+ xmlname +'.xml', outdir='K:/Rough/output2')

In [6]:
from pyroSAR.snap.auxil import gpt

from pyroSAR.snap.auxil import groupbyWorkers


groups = groupbyWorkers('K:/Rough/output/pyro11_workflow1.xml', n=1)
print(groups)

gpt(xmlfile='K:/Rough/output/pyro11_workflow1.xml', groups = groups, outdir = 'K:/Rough/output')
#gpt(xmlfile='K:/Rough/output/pyro11_workflow1.xml', outdir ='K:/Rough/output2')

[['Read', 'TOPSAR-Split'], ['Apply-Orbit-File'], ['Read (2)', 'TOPSAR-Split (2)'], ['Apply-Orbit-File (2)'], ['Back-Geocoding'], ['Enhanced-Spectral-Diversity'], ['TOPSAR-Deburst', 'Write']]
